# 1. Importing libs

In [ ]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10

In [ ]:
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

# 2. Defining our CNN

In [ ]:
# load data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

170498071/170498071 [==============================] - 2s 0us/step
(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)


## 2.1 using Sequential API

In [ ]:
def define_my_model_1(give_summary=None):
    model = keras.Sequential(
        [
            keras.Input(shape=(32, 32, 3)),
            #layer 1
            layers.Conv2D(32, 3, padding="valid", activation="relu"),
            layers.MaxPooling2D(),
            #layer 2
            layers.Conv2D(64, 3, activation="relu"),
            layers.MaxPooling2D(),
            # layer 3
            layers.Conv2D(128, 3, activation="relu"),
            layers.Flatten(),
            # layer 4
            layers.Dense(64, activation="relu"),
            # layer 5
            layers.Dense(10),
        ]
    )
    if give_summary:
      print('Sequential API Model: ')
      print(model.summary())
    else:
      pass
    return model
define_my_model_1(give_summary=True)

Sequential API Model: 
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 15, 15, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 4, 4, 128)         73856     
                                                                 
 flatten_2 (Flatten)         (N

## 2.2 using Functional API

In [ ]:
def define_my_model_2(give_summary=None):
    inputs = keras.Input(shape=(32, 32, 3))
    x1 = layers.Conv2D(32, 3)(inputs)
    x1 = layers.BatchNormalization()(x1)
    x1 = keras.activations.relu(x1)
    x1 = layers.MaxPooling2D()(x1)
    x2 = layers.Conv2D(64, 3)(x1)
    x2 = layers.BatchNormalization()(x2)
    x2 = keras.activations.relu(x2)
    x2 = layers.MaxPooling2D()(x2)
    x3 = layers.Conv2D(128, 3)(x2)
    x3 = layers.BatchNormalization()(x3)
    x3 = keras.activations.relu(x3)
    x3 = layers.Flatten()(x3)
    x4 = layers.Dense(64, activation="relu")(x3)
    outputs = layers.Dense(10)(x4)
    model = keras.Model(inputs=inputs, outputs=outputs)
    if give_summary:
      print('Functional API Model: ')
      print(model.summary())
    else:
      pass
    return model
define_my_model_2(give_summary=True)

Functional API Model: 
Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_15 (Conv2D)          (None, 30, 30, 32)        896       
                                                                 
 batch_normalization_9 (Batc  (None, 30, 30, 32)       128       
 hNormalization)                                                 
                                                                 
 tf.nn.relu_9 (TFOpLambda)   (None, 30, 30, 32)        0         
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 15, 15, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 

# 3. Training

In [ ]:
model = define_my_model_1()
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=3e-4),
    metrics=["accuracy"],
)

model.fit(x_train, y_train, batch_size=64, epochs=10, verbose=2)
model.evaluate(x_test, y_test, batch_size=64, verbose=2)

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/10
782/782 - 12s - loss: 1.7215 - accuracy: 0.3692 - 12s/epoch - 15ms/step
Epoch 2/10
782/782 - 3s - loss: 1.3909 - accuracy: 0.4999 - 3s/epoch - 4ms/step
Epoch 3/10
782/782 - 3s - loss: 1.2546 - accuracy: 0.5548 - 3s/epoch - 4ms/step
Epoch 4/10
782/782 - 3s - loss: 1.1575 - accuracy: 0.5916 - 3s/epoch - 4ms/step
Epoch 5/10
782/782 - 3s - loss: 1.0805 - accuracy: 0.6216 - 3s/epoch - 4ms/step
Epoch 6/10
782/782 - 3s - loss: 1.0160 - accuracy: 0.6448 - 3s/epoch - 4ms/step
Epoch 7/10
782/782 - 3s - loss: 0.9639 - accuracy: 0.6629 - 3s/epoch - 4ms/step
Epoch 8/10
782/782 - 3s - loss: 0.9165 - accuracy: 0.6810 - 3s/epoch - 4ms/step
Epoch 9/10
782/782 - 3s - loss: 0.8728 - accuracy: 0.6968 - 3s/epoch - 4ms/step
Epoch 10/10
782/782 - 3s - loss: 0.8337 - accuracy: 0.7120 - 3s/epoch - 3ms/step
157/157 - 1s - loss: 0.9168 - accuracy: 0.6845 - 554ms/epoch - 4ms/step


[0.9168391823768616, 0.684499979019165]

# 4. Tuning Hyperparameters/ Modifying CNN

In [ ]:
model = define_my_model_1()
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adagrad(learning_rate=3e-4),
    metrics=["accuracy"],
)

model.fit(x_train, y_train, batch_size=128, epochs=20, verbose=2)
model.evaluate(x_test, y_test, batch_size=128, verbose=2)

Epoch 1/20
391/391 - 3s - loss: 2.3040 - accuracy: 0.1013 - 3s/epoch - 7ms/step
Epoch 2/20
391/391 - 2s - loss: 2.3007 - accuracy: 0.1120 - 2s/epoch - 5ms/step
Epoch 3/20
391/391 - 2s - loss: 2.2985 - accuracy: 0.1270 - 2s/epoch - 5ms/step
Epoch 4/20
391/391 - 2s - loss: 2.2965 - accuracy: 0.1335 - 2s/epoch - 5ms/step
Epoch 5/20
391/391 - 2s - loss: 2.2944 - accuracy: 0.1521 - 2s/epoch - 5ms/step
Epoch 6/20
391/391 - 2s - loss: 2.2918 - accuracy: 0.1687 - 2s/epoch - 5ms/step
Epoch 7/20
391/391 - 2s - loss: 2.2890 - accuracy: 0.1792 - 2s/epoch - 5ms/step
Epoch 8/20
391/391 - 2s - loss: 2.2859 - accuracy: 0.1893 - 2s/epoch - 5ms/step
Epoch 9/20
391/391 - 2s - loss: 2.2824 - accuracy: 0.1928 - 2s/epoch - 5ms/step
Epoch 10/20
391/391 - 2s - loss: 2.2782 - accuracy: 0.1993 - 2s/epoch - 5ms/step
Epoch 11/20
391/391 - 2s - loss: 2.2731 - accuracy: 0.2076 - 2s/epoch - 5ms/step
Epoch 12/20
391/391 - 2s - loss: 2.2670 - accuracy: 0.2189 - 2s/epoch - 5ms/step
Epoch 13/20
391/391 - 2s - loss: 2.25

[2.135436773300171, 0.2703999876976013]

In [ ]:
def define_my_model_3(give_summary=None):
    """
      adding more layers
    """
    inputs = keras.Input(shape=(32, 32, 3))
    x1 = layers.Conv2D(32, 3)(inputs)
    x1 = layers.BatchNormalization()(x1)
    x1 = keras.activations.relu(x1)
    x1 = layers.MaxPooling2D()(x1)

    x2 = layers.Conv2D(64, 3)(x1)
    x2 = layers.BatchNormalization()(x2)
    x2 = keras.activations.relu(x2)
    # x2 = layers.MaxPooling2D()(x2)

    x3 = layers.Conv2D(128, 3)(x2)
    x3 = layers.BatchNormalization()(x3)
    x3 = keras.activations.relu(x3)
    #x3 = layers.Flatten()(x3)

    x4 = layers.Conv2D(256, 3)(x3)
    x4 = layers.BatchNormalization()(x4)
    x4 = keras.activations.relu(x4)

    x5 = layers.Conv2D(256, 3)(x4)
    x5 = layers.BatchNormalization()(x5)
    x5 = keras.activations.relu(x5)
    x5 = layers.Flatten()(x5)

    # x5 = layers.Dense(2048, activation="relu")(x4)
    x6 = layers.Dense(256, activation="relu")(x5)
    x7 = layers.Dense(128, activation="relu")(x6)
    outputs = layers.Dense(10)(x7)
    model = keras.Model(inputs=inputs, outputs=outputs)
    if give_summary:
      print('Functional API Model: ')
      print(model.summary())
    else:
      pass
    return model
define_my_model_3(give_summary=True)

Functional API Model: 
Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_49 (Conv2D)          (None, 30, 30, 32)        896       
                                                                 
 batch_normalization_36 (Bat  (None, 30, 30, 32)       128       
 chNormalization)                                                
                                                                 
 tf.nn.relu_36 (TFOpLambda)  (None, 30, 30, 32)        0         
                                                                 
 max_pooling2d_28 (MaxPoolin  (None, 15, 15, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_50 (Conv2D)          (None, 

In [ ]:
model = define_my_model_3()
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adagrad(learning_rate=3e-4),
    metrics=["accuracy"],
)

model.fit(x_train, y_train, batch_size=128, epochs=10, verbose=2)
model.evaluate(x_test, y_test, batch_size=128, verbose=2)

Epoch 1/10
391/391 - 8s - loss: 2.0475 - accuracy: 0.2544 - 8s/epoch - 20ms/step
Epoch 2/10
391/391 - 7s - loss: 1.6935 - accuracy: 0.3856 - 7s/epoch - 17ms/step
Epoch 3/10
391/391 - 7s - loss: 1.5359 - accuracy: 0.4469 - 7s/epoch - 17ms/step
Epoch 4/10
391/391 - 7s - loss: 1.4322 - accuracy: 0.4890 - 7s/epoch - 17ms/step
Epoch 5/10
391/391 - 7s - loss: 1.3552 - accuracy: 0.5188 - 7s/epoch - 17ms/step
Epoch 6/10
391/391 - 7s - loss: 1.2945 - accuracy: 0.5453 - 7s/epoch - 17ms/step
Epoch 7/10
391/391 - 7s - loss: 1.2422 - accuracy: 0.5654 - 7s/epoch - 17ms/step
Epoch 8/10
391/391 - 7s - loss: 1.1959 - accuracy: 0.5849 - 7s/epoch - 17ms/step
Epoch 9/10
391/391 - 7s - loss: 1.1557 - accuracy: 0.6002 - 7s/epoch - 17ms/step
Epoch 10/10
391/391 - 7s - loss: 1.1191 - accuracy: 0.6121 - 7s/epoch - 17ms/step
79/79 - 1s - loss: 1.2611 - accuracy: 0.5471 - 750ms/epoch - 9ms/step


[1.2611016035079956, 0.5471000075340271]

In [ ]:
model = define_my_model_3()
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=3e-4),
    metrics=["accuracy"],
)

model.fit(x_train, y_train, batch_size=128, epochs=10, verbose=2)
model.evaluate(x_test, y_test, batch_size=128, verbose=2)

Epoch 1/10
391/391 - 8s - loss: 1.2979 - accuracy: 0.5372 - 8s/epoch - 19ms/step
Epoch 2/10
391/391 - 7s - loss: 0.8674 - accuracy: 0.6941 - 7s/epoch - 17ms/step
Epoch 3/10
391/391 - 7s - loss: 0.6679 - accuracy: 0.7675 - 7s/epoch - 17ms/step
Epoch 4/10
391/391 - 7s - loss: 0.5264 - accuracy: 0.8173 - 7s/epoch - 17ms/step
Epoch 5/10
391/391 - 7s - loss: 0.3980 - accuracy: 0.8610 - 7s/epoch - 17ms/step
Epoch 6/10
391/391 - 7s - loss: 0.2875 - accuracy: 0.9005 - 7s/epoch - 17ms/step
Epoch 7/10
391/391 - 7s - loss: 0.2060 - accuracy: 0.9297 - 7s/epoch - 17ms/step
Epoch 8/10
391/391 - 7s - loss: 0.1468 - accuracy: 0.9507 - 7s/epoch - 17ms/step
Epoch 9/10
391/391 - 7s - loss: 0.1193 - accuracy: 0.9592 - 7s/epoch - 17ms/step
Epoch 10/10
391/391 - 7s - loss: 0.0966 - accuracy: 0.9669 - 7s/epoch - 17ms/step
79/79 - 1s - loss: 1.8240 - accuracy: 0.6634 - 735ms/epoch - 9ms/step


[1.8240489959716797, 0.6633999943733215]

In [ ]:
model = define_my_model_3()
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(learning_rate=3e-4),
    metrics=["accuracy"],
)

model.fit(x_train, y_train, batch_size=128, epochs=10, verbose=2)
model.evaluate(x_test, y_test, batch_size=128, verbose=2)

Epoch 1/10
391/391 - 9s - loss: 1.3922 - accuracy: 0.5042 - 9s/epoch - 22ms/step
Epoch 2/10
391/391 - 7s - loss: 0.9226 - accuracy: 0.6739 - 7s/epoch - 19ms/step
Epoch 3/10
391/391 - 7s - loss: 0.7289 - accuracy: 0.7453 - 7s/epoch - 18ms/step
Epoch 4/10
391/391 - 7s - loss: 0.5854 - accuracy: 0.7931 - 7s/epoch - 19ms/step
Epoch 5/10
391/391 - 7s - loss: 0.4709 - accuracy: 0.8326 - 7s/epoch - 18ms/step
Epoch 6/10
391/391 - 7s - loss: 0.3730 - accuracy: 0.8679 - 7s/epoch - 19ms/step
Epoch 7/10
391/391 - 7s - loss: 0.2891 - accuracy: 0.8966 - 7s/epoch - 19ms/step
Epoch 8/10
391/391 - 7s - loss: 0.2235 - accuracy: 0.9214 - 7s/epoch - 19ms/step
Epoch 9/10
391/391 - 7s - loss: 0.1772 - accuracy: 0.9378 - 7s/epoch - 19ms/step
Epoch 10/10
391/391 - 7s - loss: 0.1470 - accuracy: 0.9482 - 7s/epoch - 19ms/step
79/79 - 1s - loss: 1.1334 - accuracy: 0.7352 - 732ms/epoch - 9ms/step


[1.1334350109100342, 0.7351999878883362]

In [ ]:
# using RMSprop optimizer and new CNN model, we achieved good results
model = define_my_model_3()
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(learning_rate=3e-4),
    metrics=["accuracy"],
)

model.fit(x_train, y_train, batch_size=64, epochs=10, verbose=2)
model.evaluate(x_test, y_test, batch_size=64, verbose=2)

Epoch 1/10
782/782 - 11s - loss: 1.3252 - accuracy: 0.5303 - 11s/epoch - 14ms/step
Epoch 2/10
782/782 - 9s - loss: 0.8926 - accuracy: 0.6873 - 9s/epoch - 11ms/step
Epoch 3/10
782/782 - 9s - loss: 0.7184 - accuracy: 0.7498 - 9s/epoch - 12ms/step
Epoch 4/10
782/782 - 9s - loss: 0.6022 - accuracy: 0.7892 - 9s/epoch - 11ms/step
Epoch 5/10
782/782 - 9s - loss: 0.5047 - accuracy: 0.8205 - 9s/epoch - 11ms/step
Epoch 6/10
782/782 - 9s - loss: 0.4130 - accuracy: 0.8559 - 9s/epoch - 11ms/step
Epoch 7/10
782/782 - 9s - loss: 0.3429 - accuracy: 0.8777 - 9s/epoch - 11ms/step
Epoch 8/10
782/782 - 9s - loss: 0.2765 - accuracy: 0.9034 - 9s/epoch - 11ms/step
Epoch 9/10
782/782 - 9s - loss: 0.2299 - accuracy: 0.9207 - 9s/epoch - 11ms/step
Epoch 10/10
782/782 - 9s - loss: 0.1924 - accuracy: 0.9318 - 9s/epoch - 11ms/step
157/157 - 1s - loss: 1.1523 - accuracy: 0.7431 - 873ms/epoch - 6ms/step


[1.1522639989852905, 0.7430999875068665]

In [ ]:
# checking GPU use, just curious ;)
!nvidia-smi

Sat Jan  7 07:10:02 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P0    25W /  70W |   9090MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

You can also set the strides of the convolution, simply by adding the argument strides, for example: layers.Conv2D(32, 3, strides=(1, 1), padding='valid') and set what you want the strides to be. There are also a bunch more arguments that can be read in the docs: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D.
